In [ ]:
!pip install datasets evaluate transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/dis.experiments.4'
import sys
sys.path.append(drive_PATH)
# drive_PATH = ''

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import tqdm
import math
import numpy as np
import pandas as pd

In [ ]:
# Device for GPU speedup
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [ ]:
### MNLI Dataset ###
!pip install jsonlines
import jsonlines # jsonl imports

train_PATH = drive_PATH + '/data/multinli_1.0/multinli_1.0_train.jsonl'
dev_matched_PATH = drive_PATH + '/data/multinli_1.0/multinli_1.0_dev_matched.jsonl'
dev_mismatched_PATH = drive_PATH + '/data/multinli_1.0/multinli_1.0_dev_mismatched.jsonl'
hans_PATH = drive_PATH + '/data/hans/heuristics_evaluation_set.jsonl'

# Train Data
train_DATA = []
train_s1 = []
train_s2 = []
train_text = []
train_label = []
# Mathced Dev Data
dev_matched_DATA = []
dev_matched_s1 = []
dev_matched_s2 = []
dev_matched_text = []
dev_matched_label = []
# Mismatched Dev Data
dev_mismatched_DATA = []
dev_mismatched_s1 = []
dev_mismatched_s2 = []
dev_mismatched_text = []
dev_mismatched_label = []
# Hans Data
hans_DATA = []
hans_s1 = []
hans_s2 = []
hans_text = []
hans_label = []

with jsonlines.open(train_PATH) as f:
    for line in f.iter():
        train_DATA.append(line)
        train_s1.append(line['sentence1'])
        train_s2.append(line['sentence2'])
        train_text.append( line['sentence1'] + ' ' + line['sentence2'] )
        train_label.append(line['gold_label'])
with jsonlines.open(dev_matched_PATH) as f:
    for line in f.iter():
        dev_matched_DATA.append(line)
        dev_matched_s1.append(line['sentence1'])
        dev_matched_s2.append(line['sentence2'])
        dev_matched_text.append( line['sentence1'] + ' ' + line['sentence2'] )
        dev_matched_label.append(line['gold_label'])
with jsonlines.open(dev_mismatched_PATH) as f:
    for line in f.iter():
        dev_mismatched_DATA.append(line)
        dev_mismatched_s1.append(line['sentence1'])
        dev_mismatched_s2.append(line['sentence2'])
        dev_mismatched_text.append( line['sentence1'] + ' ' + line['sentence2'] )
        dev_mismatched_label.append(line['gold_label'])
with jsonlines.open(hans_PATH) as f:
    for line in f.iter():
        hans_DATA.append(line)
        hans_s1.append(line['sentence1'])
        hans_s2.append(line['sentence2'])
        hans_text.append( line['sentence1'] + ' ' + line['sentence2'] )
        hans_label.append(line['gold_label'])

### Cleaning Datasets

# Train
train_label = np.array(train_label, dtype='<U14')
train_s1 = np.array(train_s1)
train_s2 = np.array(train_s2)
train_label[(train_label == 'neutral') | (train_label == 'contradiction')] = 'non-entailment'
train_label[train_label == ['entailment']] = 1
train_label[train_label == ['non-entailment']] = 0
train_label = np.array(train_label, dtype='int')

# Dev Matched
dev_matched_label = np.array(dev_matched_label, dtype='<U14')
dev_matched_filter = dev_matched_label != '-'
dev_matched_s1 = np.array(dev_matched_s1)[dev_matched_filter]
dev_matched_s2 = np.array(dev_matched_s2)[dev_matched_filter]
dev_matched_label = dev_matched_label[dev_matched_filter]
dev_matched_label[(dev_matched_label == 'neutral') | (dev_matched_label == 'contradiction')] = 'non-entailment'
dev_matched_label[dev_matched_label == ['entailment']] = 1
dev_matched_label[dev_matched_label == ['non-entailment']] = 0
dev_matched_label = np.array(dev_matched_label, dtype='int')

# Dev Mismatched
dev_mismatched_label = np.array(dev_mismatched_label, dtype='<U14')
dev_mismatched_filter = dev_mismatched_label != '-'
dev_mismatched_s1 = np.array(dev_mismatched_s1)[dev_mismatched_filter]
dev_mismatched_s2 = np.array(dev_mismatched_s2)[dev_mismatched_filter]
dev_mismatched_label = dev_mismatched_label[dev_mismatched_filter]
dev_mismatched_label[(dev_mismatched_label == 'neutral') | (dev_mismatched_label == 'contradiction')] = 'non-entailment'
dev_mismatched_label[dev_mismatched_label == ['entailment']] = 1
dev_mismatched_label[dev_mismatched_label == ['non-entailment']] = 0
dev_mismatched_label = np.array(dev_mismatched_label, dtype='int')

# HANS
hans_label = np.array(hans_label)
hans_s1 = np.array(hans_s1)
hans_s2 = np.array(hans_s2)
hans_label[hans_label == ['entailment']] = 1
hans_label[hans_label == ['non-entailment']] = 0
hans_label = np.array(hans_label, dtype='int')

train_labels = np.unique(train_label)
dev_matched_labels = np.unique(dev_matched_label)
dev_mismatched_labels = np.unique(dev_mismatched_label)
hans_labels = np.unique(np.array(hans_label))

value_counts = pd.concat({'train_label' : pd.DataFrame(train_label).value_counts(),
                        'dev_matched_label' : pd.DataFrame(dev_matched_label).value_counts(),
                        'dev_mismatched_label' : pd.DataFrame(dev_mismatched_label).value_counts(),
                        'hans_label' : pd.DataFrame(hans_label).value_counts()})

### Balancing Act
def balanced_idx(label_dataset):
    idx1 = np.array(range(len(label_dataset)))[label_dataset == 1]
    idx0 = np.array(range(len(label_dataset)))[label_dataset == 0]
    idx0_selected_i = np.random.choice(idx0.shape[0], len(idx1), replace=False)
    idx0_selected = idx0[idx0_selected_i]
    idx = np.concatenate((idx1, idx0_selected))
    np.random.shuffle(idx) # random shuffle
    return idx

# Balancing Train
train_balanced_idx = balanced_idx(train_label)
train_s1 = train_s1[train_balanced_idx]
train_s2 = train_s2[train_balanced_idx]
train_label = train_label[train_balanced_idx]

# Balancing Dev Matched
dev_matched_balanced_idx = balanced_idx(dev_matched_label)
dev_matched_s1 = dev_matched_s1[dev_matched_balanced_idx]
dev_matched_s2 = dev_matched_s2[dev_matched_balanced_idx]
dev_matched_label = dev_matched_label[dev_matched_balanced_idx]

# Balancing Dev Mismatched
dev_mismatched_balanced_idx = balanced_idx(dev_mismatched_label)
dev_mismatched_s1 = dev_mismatched_s1[dev_mismatched_balanced_idx]
dev_mismatched_s2 = dev_mismatched_s2[dev_mismatched_balanced_idx]
dev_mismatched_label = dev_mismatched_label[dev_mismatched_balanced_idx]

# Balancing HANS (already balanced)
hans_balanced_idx = balanced_idx(hans_label)
hans_s1 = hans_s1[hans_balanced_idx]
hans_s2 = hans_s2[hans_balanced_idx]
hans_label = hans_label[hans_balanced_idx]

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#####################
### Training Data ###
#####################
train_input_ids = []
train_attention_masks = []
train_token_type_ids = []
len_train = len(train_s1)
assert ((len(train_s1) == len_train) and (len(train_s2) == len_train))
for i in tqdm.tqdm(range(len_train)):
  encoded_dict = tokenizer.encode_plus(
                      train_s1[i], train_s2[i],
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 512,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                  )
  train_input_ids.append(encoded_dict['input_ids'])
  train_attention_masks.append(encoded_dict['attention_mask'])
  train_token_type_ids.append(encoded_dict['token_type_ids'])
# Convert the lists into tensors
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_token_type_ids = torch.cat(train_token_type_ids, dim=0)
train_labels = torch.tensor(train_label)

##########################
### Dev (Matched) Data ###
##########################
dev_matched_input_ids = []
dev_matched_attention_masks = []
dev_matched_token_type_ids = []
len_dev_matched = len(dev_matched_s1)
assert ((len(dev_matched_s1) == len_dev_matched) and (len(dev_matched_s2) == len_dev_matched))
for i in tqdm.tqdm(range(len_dev_matched)):
  encoded_dict = tokenizer.encode_plus(
                      dev_matched_s1[i], dev_matched_s2[i],
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 512,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                  )
  dev_matched_input_ids.append(encoded_dict['input_ids'])
  dev_matched_attention_masks.append(encoded_dict['attention_mask'])
  dev_matched_token_type_ids.append(encoded_dict['token_type_ids'])
# Convert the lists into tensors
dev_matched_input_ids = torch.cat(dev_matched_input_ids, dim=0)
dev_matched_attention_masks = torch.cat(dev_matched_attention_masks, dim=0)
dev_matched_token_type_ids = torch.cat(dev_matched_token_type_ids, dim=0)
dev_matched_labels = torch.tensor(dev_matched_label)

#############################
### Dev (Mismatched) Data ###
#############################
dev_mismatched_input_ids = []
dev_mismatched_attention_masks = []
dev_mismatched_token_type_ids = []
len_dev_mismatched = len(dev_mismatched_s1)
assert ((len(dev_mismatched_s1) == len_dev_mismatched) and (len(dev_mismatched_s2) == len_dev_mismatched))
for i in tqdm.tqdm(range(len_dev_mismatched)):
  encoded_dict = tokenizer.encode_plus(
                      dev_mismatched_s1[i], dev_mismatched_s2[i],
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 512,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                  )
  dev_mismatched_input_ids.append(encoded_dict['input_ids'])
  dev_mismatched_attention_masks.append(encoded_dict['attention_mask'])
  dev_mismatched_token_type_ids.append(encoded_dict['token_type_ids'])
# Convert the lists into tensors
dev_mismatched_input_ids = torch.cat(dev_mismatched_input_ids, dim=0)
dev_mismatched_attention_masks = torch.cat(dev_mismatched_attention_masks, dim=0)
dev_mismatched_token_type_ids = torch.cat(dev_mismatched_token_type_ids, dim=0)
dev_mismatched_labels = torch.tensor(dev_mismatched_label)

#################
### HANS Data ###
#################
hans_input_ids = []
hans_attention_masks = []
hans_token_type_ids = []
len_hans = len(hans_s1)
assert ((len(hans_s1) == len_hans) and (len(hans_s2) == len_hans))
for i in tqdm.tqdm(range(len_hans)):
  encoded_dict = tokenizer.encode_plus(
                      hans_s1[i], hans_s2[i],
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 512,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                      )
  hans_input_ids.append(encoded_dict['input_ids'])
  hans_attention_masks.append(encoded_dict['attention_mask'])
  hans_token_type_ids.append(encoded_dict['token_type_ids'])
# Convert the lists into tensors
hans_input_ids = torch.cat(hans_input_ids, dim=0)
hans_attention_masks = torch.cat(hans_attention_masks, dim=0)
hans_token_type_ids = torch.cat(hans_token_type_ids, dim=0)
hans_labels = torch.tensor(hans_label)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

  0%|          | 0/261798 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 30000/30000 [00:24<00:00, 1201.10it/s]


In [20]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

### Datasets ###
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_token_type_ids, train_labels)
dev_matched_dataset = TensorDataset(dev_matched_input_ids, dev_matched_attention_masks, dev_matched_token_type_ids, dev_matched_labels)
dev_mismatched_dataset = TensorDataset(dev_mismatched_input_ids, dev_mismatched_attention_masks, dev_mismatched_token_type_ids, dev_mismatched_labels)
hans_dataset = TensorDataset(hans_input_ids, hans_attention_masks, hans_token_type_ids , hans_labels)
### Dataset Splits ###
# MNLI
num_train = int(len(train_dataset) * 0.95)
num_valid = len(train_dataset) - num_train #int(len(train_dataset) * 0.05)
split_train, split_valid = random_split( train_dataset, [num_train, num_valid] )
split_test_matched = dev_matched_dataset
split_test_mismatched = dev_mismatched_dataset
# HANS
num_train_hans = int(len(hans_dataset) * 0.80)
num_valid_hans = int(len(hans_dataset) * 0.10)
num_test_hans = len(hans_dataset) - num_train_hans - num_valid_hans #int(len(hans_dataset) * 0.10)
split_train_hans , split_valid_hans, split_test_hans = random_split( hans_dataset, [num_train_hans, num_valid_hans , num_test_hans] )
# Final Datasets
# MNLI  # Training: split_train_ & split_valid_
        # Testing: dev_matched_dataset & dev_mismatched_dataset
# HANS  # Training: split_train_train_hans_ & split_train_valid_hans_
        # Testing: split_test_hans_

In [21]:
import transformers

#########################
### Define the Models ###
#########################
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear(768, 2)

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        # softmax_output = torch.nn.functional.softmax(output, dim=1)
        return output

class BERTClass_Adv(torch.nn.Module):
    def __init__(self):
        super(BERTClass_Adv, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear(768, 2)

    def forward(self, ids, mask, token_type_ids):
        # Only use the second sentence
        ids = ids.clone().detach()
        mask = mask.clone().detach()
        token_type_ids = token_type_ids.clone().detach()
        ids[token_type_ids < 1] = 0
        mask[token_type_ids < 1] = 0
        token_type_ids[token_type_ids < 1] = 0
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        # softmax_output = torch.nn.functional.softmax(output, dim=1)
        return output

### POE LOSS
import torch.nn.functional as F
class POELoss(nn.Module):
    """Implements the product of expert loss."""
    def __init__(self, poe_alpha=1.0):
        super().__init__()
        self.poe_alpha = poe_alpha

    def forward(self, inputs, targets, inputs_adv):
        targets = targets.view(-1, 1)
        # print('targets:', targets)
        pt = F.softmax(inputs, dim=1)
        # print('pt:', pt)
        pt_adv = F.softmax(inputs_adv, dim=1)
        # print('pt_adv:', pt_adv)
        joint_pt = F.softmax((torch.log(pt) + self.poe_alpha*torch.log(pt_adv)), dim=1)
        # print('joint_pt:', joint_pt)
        joint_p = joint_pt.gather(1, targets)
        # print('joint_p:', joint_p)
        batch_loss = -torch.log(joint_p)
        # print('batch_loss:', batch_loss)
        loss = batch_loss.mean()
        # print('loss:', loss)
        return loss

"""
Main module for Debiasing.
"""
class DebiasNet(nn.Module):
    """ This module wrap the NLI model and applied the debiasing technique to it."""
    def __init__(self, class_model, class_model_adv):
        super(DebiasNet, self).__init__()
        # Loss options
        self.loss_fct_poe = POELoss()
        self.loss_fct_ce = torch.nn.CrossEntropyLoss()
        # Models
        self.class_model = class_model
        self.class_model_adv = class_model_adv

    def forward(self, ids, mask, token_type_ids, labels):
        # Outputs
        output_class_model = self.class_model(ids, mask, token_type_ids)
        output_class_model_adv = self.class_model_adv(ids, mask, token_type_ids)
        # Losses
        loss_poe = self.loss_fct_poe(output_class_model, labels, output_class_model_adv)
        loss_ce = self.loss_fct_ce(output_class_model, labels)
        outputs = {}
        outputs['output_class_model'] = output_class_model
        outputs['output_class_model_adv'] = output_class_model_adv
        outputs['loss_poe'] = loss_poe
        outputs['loss_ce'] = loss_ce
        return outputs

In [22]:
###############################
### Define the Data Loaders ###
###############################
BATCH_SIZE = 16
# MNLI
train_dataloader = DataLoader(split_train, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(split_valid, batch_size=BATCH_SIZE, shuffle=True)
test_matched_dataloader = DataLoader(split_test_matched, batch_size=BATCH_SIZE, shuffle=True)
test_mismatched_dataloader = DataLoader(split_test_mismatched, batch_size=BATCH_SIZE, shuffle=True)
# HANS
hans_train_dataloader = DataLoader(split_train_hans, batch_size=BATCH_SIZE, shuffle=True)
hans_valid_dataloader = DataLoader(split_valid_hans, batch_size=BATCH_SIZE, shuffle=True)
hans_test_dataloader = DataLoader(split_test_hans, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
## Train the Adverserial Model

# Initialize the Adverserial Models
model_class_adv = BERTClass_Adv()
model_class_adv.to(DEVICE)
################################
### Define the Loss Function ###
################################
loss_function = torch.nn.CrossEntropyLoss()
############################
### Define the Optimizer ###
############################
LEARNING_RATE =  2e-5
optimizer = torch.optim.AdamW(model_class_adv.parameters(), lr=LEARNING_RATE)
##################################
### Define the Evaluation Loop ###
##################################
def evaluate(dataloader):
    model_class_adv.cuda()
    model_class_adv.eval()
    total_loss, total_acc, total_count, total_count_n = 0, 0, 0, 0
    with torch.no_grad():
      for step, batch in tqdm.tqdm(enumerate(dataloader)):
          # 0. Put ids & mask & labels to the device
          b_input_ids = batch[0].to(DEVICE)
          b_input_mask = batch[1].to(DEVICE)
          b_token_type_ids = batch[2].to(DEVICE)
          b_labels = batch[3].to(DEVICE)
          # 2. Forward Pass
          output = model_class_adv(b_input_ids, b_input_mask, b_token_type_ids)
          logits = output
          predicted_class_labels = logits.argmax(dim=1)
          # 3. Loss
          valid_loss = loss_function(logits, b_labels)
          valid_acc = (predicted_class_labels == b_labels).sum().item()
          total_loss += valid_loss
          total_acc += valid_acc
          total_count += len(b_labels)
          total_count_n += 1
    Loss = total_loss / total_count_n
    Acc = total_acc / total_count
    return Loss, Acc

# TRAINING & VALIDATION
import time
EPOCHS = 1

model_paths = []
# MNLI Losses
train_losses, train_accs = [], []
val_losses, val_accs = [], []
test_matched_losses, test_matched_accs = [], []
test_mismatched_losses, test_mismatched_accs = [], []
# HANS Losses
hans_train_losses , hans_train_accs  = [], []
hans_valid_losses, hans_valid_accs  = [], []
hans_test_losses, hans_test_accs = [], []
# Model ID
model_id = '-'.join(time.ctime(time.time()).replace(':', ' ').split(' ')[2:5])

### EPOCHS ###
for epoch in range(1, EPOCHS + 1):
    ################
    ### TRAINING ###
    ################
    epoch_start_time = time.time()
    ############################################################################
    model_class_adv.cuda()
    model_class_adv.train()
    total_loss, total_acc, total_count = 0, 0, 0
    log_interval = 100
    dataloader = train_dataloader # CHANGE TO TRAIN DATALOADER
    for step, batch in tqdm.tqdm(enumerate(dataloader)):
        # 0. Put ids & mask & labels to the device
        b_input_ids = batch[0].to(DEVICE)
        b_input_mask = batch[1].to(DEVICE)
        b_token_type_ids = batch[2].to(DEVICE)
        b_labels = batch[3].to(DEVICE)
        # 1. Clear Gradient
        optimizer.zero_grad()
        # 2. Forward Pass
        output = model_class_adv(b_input_ids, b_input_mask, b_token_type_ids)
        logits = output                        # predicted probabilities
        predicted_class_labels = logits.argmax(dim=1) # predicted classes
        # 3. Loss
        train_loss = loss_function(logits, b_labels)
        # 4. Backward Pass
        train_loss.backward()
        # 5. Optimizer Step
        optimizer.step()
        # 6. Progress Display
        train_acc = (predicted_class_labels == b_labels).sum().item() / len(b_labels)
        total_loss += train_loss.item()
        total_acc += train_acc
        total_count += 1
        if step % log_interval == 0 and step > 0:
            loss = total_loss / total_count
            acc = total_acc / total_count
            train_losses.append(loss)
            train_accs.append(acc)
            print("| epoch {: d} | {:5d}/{:5d} batches "
                  "| loss {:8.3f}| accuracy {:8.3f}".format(epoch, step, len(dataloader),loss , acc))
            total_loss, total_acc, total_count = 0, 0, 0
    ############################################################################
    epoch_end_time = time.time()
    training_time = epoch_end_time - epoch_start_time
    print("-" * 59)
    print("| end of epoch {:3d} | time: {:5.2f}s |".format( epoch, training_time))
    print("-" * 59)
    ##################
    ### VALIDATION ###
    ##################
    # Training
    loss_train = train_losses[-1]
    acc_train = train_accs[-1]
    # Validation
    loss_val, acc_val = evaluate(valid_dataloader)
    val_losses.append(loss_val)
    val_accs.append(acc_val)
    # Test Matched
    loss_test_matched, acc_test_matched = evaluate(test_matched_dataloader)
    test_matched_losses.append(loss_test_matched)
    test_matched_accs.append(acc_test_matched)
    # Test Mismatched
    loss_test_mismatched, acc_test_mismatched = evaluate(test_mismatched_dataloader)
    test_mismatched_losses.append(loss_test_mismatched)
    test_mismatched_accs.append(acc_test_mismatched)
    # Hans Validation
    loss_hans_valid, acc_hans_valid = evaluate(hans_valid_dataloader)
    hans_valid_losses.append(loss_hans_valid)
    hans_valid_accs.append(acc_hans_valid)
    # Hans Test
    loss_hans_test , acc_hans_test = evaluate(hans_test_dataloader)
    hans_test_losses.append(loss_hans_test)
    hans_test_accs.append(acc_hans_test)
    # Register model path
    model_paths.append(f'id-{model_id}-epoch-{epoch}-acc_train-{acc_train:.2f}-acc_val-{acc_val:.2f}.pt')
    # Save model to path
    torch.save(model_class_adv.state_dict(), drive_PATH+'/model_states/'+model_paths[-1])
    # Display Results
    print("| Validation Loss {:8.3f} | Validation Accuracy {:8.3f}".format( loss_val, acc_val))
    print("| Test Matched Loss {:8.3f} | Test Matched Accuracy {:8.3f}".format( loss_test_matched, acc_test_matched))
    print("| Test Mismatched Loss {:8.3f} | Test Mismatched Accuracy {:8.3f}".format( loss_test_mismatched, acc_test_mismatched))
    print("| HANS Validation Loss {:8.3f} | HANS Validation Accuracy {:8.3f}".format( loss_hans_valid, acc_hans_valid))
    print("| HANS Test Loss {:8.3f} | HANS Test Accuracy {:8.3f}".format( loss_hans_test, acc_hans_test))
    print("-" * 59)

101it [00:35,  2.82it/s]

| epoch  1 |   100/15545 batches | loss    0.692| accuracy    0.551


201it [01:11,  2.83it/s]

| epoch  1 |   200/15545 batches | loss    0.664| accuracy    0.589


301it [01:46,  2.83it/s]

...

| epoch  1 | 15500/15545 batches | loss    0.588| accuracy    0.681


15545it [1:31:32,  2.83it/s]


-----------------------------------------------------------
| end of epoch   1 | time: 5492.53s |
-----------------------------------------------------------


819it [01:33,  8.72it/s]
435it [00:50,  8.69it/s]
433it [00:49,  8.70it/s]
188it [00:21,  8.72it/s]
188it [00:21,  8.72it/s]


| Validation Loss    0.584 | Validation Accuracy    0.690
| Test Matched Loss    0.578 | Test Matched Accuracy    0.693
| Test Mismatched Loss    0.576 | Test Mismatched Accuracy    0.695
| HANS Validation Loss    0.760 | HANS Validation Accuracy    0.502
| HANS Test Loss    0.768 | HANS Test Accuracy    0.494
-----------------------------------------------------------


In [ ]:
### FREEZING ALL LAYERS
for p in model_class_adv.parameters():
    p.requires_grad = False
# Initialize the Classification Model
model_class = BERTClass()
model_class.to(DEVICE)
model_debias = DebiasNet(model_class , model_class_adv)
model_debias.to(DEVICE)
############################
### Define the Optimizer ###
############################
from transformers import AdamW
LEARNING_RATE =  2e-5
optimizer = AdamW(model_debias.parameters(), lr=LEARNING_RATE)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
##################################
### Define the Evaluation Loop ###
##################################
def evaluate(dataloader):
    model_debias.cuda()
    model_debias.eval()
    total_loss, total_acc, total_count, total_count_n = 0, 0, 0, 0
    with torch.no_grad():
      for step, batch in tqdm.tqdm(enumerate(dataloader)):
          # 0. Put ids & mask & labels to the device
          b_input_ids = batch[0].to(DEVICE)
          b_input_mask = batch[1].to(DEVICE)
          b_token_type_ids = batch[2].to(DEVICE)
          b_labels = batch[3].to(DEVICE)
          # 2. Forward Pass
          output = model_debias(b_input_ids, b_input_mask, b_token_type_ids, b_labels)
          logits = output['output_class_model']
          predicted_class_labels = logits.argmax(dim=1)
          # 3. Loss
          valid_loss = output['loss_poe']
          valid_acc = (predicted_class_labels == b_labels).sum().item()
          total_loss += valid_loss
          total_acc += valid_acc
          total_count += len(b_labels)
          total_count_n += 1
    Loss = total_loss / total_count_n
    Acc = total_acc / total_count
    return Loss, Acc

In [ ]:
# TRAINING & VALIDATION
import time
EPOCHS = 1

model_paths = []
# MNLI Losses
train_losses, train_accs = [], []
val_losses, val_accs = [], []
test_matched_losses, test_matched_accs = [], []
test_mismatched_losses, test_mismatched_accs = [], []
# HANS Losses
hans_train_losses , hans_train_accs  = [], []
hans_valid_losses, hans_valid_accs  = [], []
hans_test_losses, hans_test_accs = [], []
# Model ID
model_id = '-'.join(time.ctime(time.time()).replace(':', ' ').split(' ')[2:5])

### EPOCHS ###
for epoch in range(1, EPOCHS + 1):
    ################
    ### TRAINING ###
    ################
    epoch_start_time = time.time()
    ############################################################################
    model_debias.cuda()
    model_debias.train()
    total_loss, total_acc, total_count = 0, 0, 0
    log_interval = 100
    dataloader = train_dataloader # CHANGE TO TRAIN DATALOADER
    for step, batch in tqdm.tqdm(enumerate(dataloader)):
        # 0. Put ids & mask & labels to the device
        b_input_ids = batch[0].to(DEVICE)
        b_input_mask = batch[1].to(DEVICE)
        b_token_type_ids = batch[2].to(DEVICE)
        b_labels = batch[3].to(DEVICE)
        # 1. Clear Gradient
        optimizer.zero_grad()
        # 2. Forward Pass
        output = model_debias(b_input_ids, b_input_mask, b_token_type_ids, b_labels)
        logits = output['output_class_model']         # predicted probabilities
        predicted_class_labels = logits.argmax(dim=1) # predicted classes
        # 3. Loss
        train_loss = output['loss_poe']
        # 4. Backward Pass
        train_loss.backward()
        # 5. Optimizer Step
        optimizer.step()
        # 6. Progress Display
        train_acc = (predicted_class_labels == b_labels).sum().item() / len(b_labels)
        total_loss += train_loss.item()
        total_acc += train_acc
        total_count += 1
        if step % log_interval == 0 and step > 0:
            loss = total_loss / total_count
            acc = total_acc / total_count
            train_losses.append(loss)
            train_accs.append(acc)
            print("| epoch {: d} | {:5d}/{:5d} batches "
                  "| loss {:8.3f}| accuracy {:8.3f}".format(epoch, step, len(dataloader),loss , acc))
            total_loss, total_acc, total_count = 0, 0, 0
    ############################################################################
    epoch_end_time = time.time()
    training_time = epoch_end_time - epoch_start_time
    print("-" * 59)
    print("| end of epoch {:3d} | time: {:5.2f}s |".format( epoch, training_time))
    print("-" * 59)
    ##################
    ### VALIDATION ###
    ##################
    # Training
    loss_train = train_losses[-1]
    acc_train = train_accs[-1]
    # Validation
    loss_val, acc_val = evaluate(valid_dataloader)
    val_losses.append(loss_val)
    val_accs.append(acc_val)
    # Test Matched
    loss_test_matched, acc_test_matched = evaluate(test_matched_dataloader)
    test_matched_losses.append(loss_test_matched)
    test_matched_accs.append(acc_test_matched)
    # Test Mismatched
    loss_test_mismatched, acc_test_mismatched = evaluate(test_mismatched_dataloader)
    test_mismatched_losses.append(loss_test_mismatched)
    test_mismatched_accs.append(acc_test_mismatched)
    # Hans Validation
    loss_hans_valid, acc_hans_valid = evaluate(hans_valid_dataloader)
    hans_valid_losses.append(loss_hans_valid)
    hans_valid_accs.append(acc_hans_valid)
    # Hans Test
    loss_hans_test , acc_hans_test = evaluate(hans_test_dataloader)
    hans_test_losses.append(loss_hans_test)
    hans_test_accs.append(acc_hans_test)
    # Register model path
    model_paths.append(f'id-{model_id}-epoch-{epoch}-acc_train-{acc_train:.2f}-acc_val-{acc_val:.2f}.pt')
    # Save model to path
    torch.save(model_debias.state_dict(), drive_PATH+'/model_states/'+model_paths[-1])
    # Display Results
    print("| Validation Loss {:8.3f} | Validation Accuracy {:8.3f}".format( loss_val, acc_val))
    print("| Test Matched Loss {:8.3f} | Test Matched Accuracy {:8.3f}".format( loss_test_matched, acc_test_matched))
    print("| Test Mismatched Loss {:8.3f} | Test Mismatched Accuracy {:8.3f}".format( loss_test_mismatched, acc_test_mismatched))
    print("| HANS Validation Loss {:8.3f} | HANS Validation Accuracy {:8.3f}".format( loss_hans_valid, acc_hans_valid))
    print("| HANS Test Loss {:8.3f} | HANS Test Accuracy {:8.3f}".format( loss_hans_test, acc_hans_test))
    print("-" * 59)

101it [00:47,  2.12it/s]

| epoch  1 |   100/15545 batches | loss    0.513| accuracy    0.631


201it [01:34,  2.12it/s]

| epoch  1 |   200/15545 batches | loss    0.462| accuracy    0.684


301it [02:21,  2.12it/s]

...

| epoch  1 | 15500/15545 batches | loss    0.270| accuracy    0.864


15545it [2:02:02,  2.12it/s]


-----------------------------------------------------------
| end of epoch   1 | time: 7322.32s |
-----------------------------------------------------------


819it [03:05,  4.41it/s]
435it [01:38,  4.41it/s]
433it [01:38,  4.40it/s]
188it [00:42,  4.41it/s]
188it [00:42,  4.41it/s]


| Validation Loss    0.285 | Validation Accuracy    0.874
| Test Matched Loss    0.268 | Test Matched Accuracy    0.887
| Test Mismatched Loss    0.258 | Test Mismatched Accuracy    0.887
| HANS Validation Loss    1.921 | HANS Validation Accuracy    0.518
| HANS Test Loss    1.966 | HANS Test Accuracy    0.507
-----------------------------------------------------------


In [ ]:
# # Loss
# loss_fct = POELoss()
# # Model
# model = BERTClass_Adv()
# # model.to(DEVICE)
# # Data
# BATCH_SIZE = 4
# valid_dataloader = DataLoader(split_valid, batch_size=BATCH_SIZE, shuffle=True)
# dataloader = valid_dataloader
# for step, batch in tqdm.tqdm(enumerate(dataloader)):
#     b_input_ids = batch[0].to(DEVICE)
#     b_input_mask = batch[1].to(DEVICE)
#     b_token_type_ids = batch[2].to(DEVICE)
#     b_labels = batch[3].to(DEVICE)
#     break

# # Test the forward method with the generated data
# with torch.no_grad():
#     output = model_class(b_input_ids, b_input_mask, b_token_type_ids)
#     output_adv = model_class_adv(b_input_ids, b_input_mask, b_token_type_ids)

# output, output_adv, loss_fct(output, b_labels, output_adv)

# model_class = BERTClass()
# model_class_adv = BERTClass_Adv()
# model_class.to(DEVICE)
# model_class_adv.to(DEVICE)
# model = DebiasNet(model_class , model_class_adv)

# # Data
# BATCH_SIZE = 1
# valid_dataloader = DataLoader(split_valid, batch_size=BATCH_SIZE, shuffle=True)
# dataloader = valid_dataloader
# for step, batch in tqdm.tqdm(enumerate(dataloader)):
#     b_input_ids = batch[0].to(DEVICE)
#     b_input_mask = batch[1].to(DEVICE)
#     b_token_type_ids = batch[2].to(DEVICE)
#     b_labels = batch[3].to(DEVICE)
#     break

# # Test the forward method with the generated data
# with torch.no_grad():
#     output = model(b_input_ids, b_input_mask, b_token_type_ids, b_labels)

# output